# Hands On Session: Dimensionality Reduction, Principal Components Analysis (PCA), and Singular Value Decomposition (SVD)
# By: Sabera Talukder

In [71]:
# All Imports - alphabetically ordered with shortcuts
import matplotlib.pyplot as plt
import numpy as np
import time

from mpl_toolkits import mplot3d
from numpy.linalg import svd
from scipy.io import loadmat
from sklearn.decomposition import PCA

# Data Exploration

## Hint: do not reinvent the wheel! If you want to do something, a preexisiting package, library, function, etc. exists to do what you want. Google & Stack Overflow are your friends 😃

## Load Data
### Dataset background: today we'll be working with calcium imaging data from one male mouse. We have already converted the calcium imaging videos into continuous neural signals, so you don't have to worry about it (you're welcome 😘). The male mouse has different visitors in his cage throughout the recording, and we'll explore dimensionality reduction by determining if it's 🐭❤️ or 🐭 😡!

#### Let's start by loading in our dataset!

In [72]:
hypothalamus_data = loadmat('../../data/hypothalamus_calcium_imaging_remedios_et_al.mat')

## How many data arrays are contained in hypothalamus_data?
#### Hint: what happens if you type the variable name in a cell and run the cell?

In [73]:
# Enter code here:
hypothalamus_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'attack_vector', 'neural_data', 'sex_vector'])

In [74]:
hypothalamus_data['attack_vector'].shape, hypothalamus_data['sex_vector'].shape, hypothalamus_data['neural_data'].shape

((1, 18561), (1, 18561), (115, 18561))

## Extract the N data arrays into N separate variables.

In [75]:
# Enter code here:
attack = hypothalamus_data['attack_vector']
sex = hypothalamus_data['sex_vector']
robs = hypothalamus_data['neural_data']

## What is the dimensionality of each of the N data arrays?
## What do you think the dimensions represent?

In [76]:
# Enter code here:
attack.shape, sex.shape, robs.shape

# The attack vector is probably whether there was an attack or not for each timepoint.
# The attack vector is probably sex for each timepoint.
# The neural data is probably the neuron response for each timepoint for each of the 115 cells.

((1, 18561), (1, 18561), (115, 18561))

## Visualize the distributions of each of the N data arrays as a histogram!
#### Hint: the answer to this question can be a picture!
#### Hint Hint: sometimes functions run faster if you transform a matrix a vector first.

In [77]:
np.histogram(attack, bins=2)[0]

array([17454,  1107])

In [78]:
# Enter code here:
plt.subplot(3,1,1)
# barchart for the histogram
# for some reason, plt.hist() is suupppeeerrr slow compared to np.histogram()
plt.bar([0,1], np.histogram(attack, bins=2)[0])
# use only ticks for 0 and 1, there is no point to interpolate
plt.xticks([0,1])
plt.title("attack")
plt.subplot(3,1,2)
plt.bar([0,1], np.histogram(sex, bins=2)[0])
plt.xticks([0,1])
plt.title("sex")
plt.subplot(3,1,3)
plt.plot(np.histogram(robs[0,:])[0], '-o')
plt.title("robs, cell 0")

/var/folders/rc/lxb7w_kn1gd1gppq2y1hq2tm0000gn/T/ipykernel_13092/53606190.py:2: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(3,1,1)


Text(0.5, 1.0, 'robs, cell 0')

## Plot the N data arrays.
#### Hint: sometimes the most expeditious way to visualize data is to treat it as an image!
#### Hint Hint: one visualization might give you something you dont expect, but is the problem the data?

In [79]:
attack.shape[1], attack.shape[1]/3

(18561, 6187.0)

In [80]:
attack.reshape((6187, 3))

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint8)

In [81]:
# Enter code here:
plt.subplot(1,3,1)
# up to the last timepoint, since it is easy to reshape this way
plt.imshow(attack[0,0:-1].reshape((116, 160)), interpolation=None)
plt.title("attack")

plt.subplot(1,3,2)
plt.imshow(sex[0,0:-1].reshape((116, 160)), interpolation=None)
plt.title("sex")
plt.subplot(1,3,3)
plt.imshow(robs[0,0:-1].reshape((116,160)))
plt.title("robs, cell 0")

/var/folders/rc/lxb7w_kn1gd1gppq2y1hq2tm0000gn/T/ipykernel_13092/286686699.py:2: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(1,3,1)


Text(0.5, 1.0, 'robs, cell 0')

## What do the values inside the arrays represent?

In [82]:
# Enter answer here (code can be used, but not required):
# The attack vector is probably whether there was an attack or not for each timepoint.
# The attack vector is probably sex for each timepoint.
# The neural data is probably the neuron response for each timepoint for each of the 115 cells.

### Great job exploring the data! Now let's dive into what we can do with it!

# Dimensionality Reduction

## Principal Components Analysis (PCA)
#### We're going to dive into how PCA works, but first we're going to see what can be done with it! All you need to know for now is that PCA creates a lower dimensional representation of your data to preserve the data's variance.

#### By now you know you have a neural data array that is number_of_neurons by time, let's say the dimensionality is NxT. What we are going to do with PCA is take all of our time steps and compress them; this will output an array that is SxT where S < N. In other words each time step is initially an N dimensional vector, that gets compressed into an S dimensional vector where S < N. Let's explore this with S = 3.

In [83]:
# make a PCA model with S = 3
pca_model_s_3 = PCA(n_components=3)

# STOP & Check Yourself: Do you know why we can just call "PCA"?
# yes, this initializes the PCA object with 3 dimensions

# with the PCA model instance we created to our neural data
neural_pca_s_3 = pca_model_s_3.fit_transform(robs.T)

## What is the dimensionality of the PCAed neural data? What do these dimensions mean?

In [84]:
# Enter code here:
neural_pca_s_3.shape

# T x L
# T = number of timepoints
# L = number of latents (principle components)

(18561, 3)

## Plot the Principal Components (aka PCs) in 3D!

In [85]:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')

# Enter code here:
pc1 = neural_pca_s_3[:,0]
pc2 = neural_pca_s_3[:,1]
pc3 = neural_pca_s_3[:,2]

ax.scatter3D(pc1, pc2, pc3);

<IPython.core.display.Javascript object>

## Nice job! Now rotate your representation! What interesting things do you notice about your dimensionality reduced data?
#### Hint: why does this data look connected?
#### Hint: Why are the axes so different from each other? What do they represent?

In [86]:
# Enter answer here:
# the big "split" in the data is probably related to sex
# (the size of each arm looks related to sex)
# the up-down variation on each arm is probably related to attack/noattack

In [87]:
# do this to switch out of movable 3d plotting (i.e. when you have 2d plots next)
%matplotlib inline 

## We're going to return to this visualization, but first you have to be thinking to yourself, we got rid of A LOT of dimensions how do we know this representation is still good? Great question! You tell me 👇🏻👇🏼👇🏽👇🏾👇🏿

- if the components capture the majority of the variance,
- then they are a good representation of the data

## How much variance is explained by each of these top 3 principal components? What does this tell you about the data?
#### Hint: what is the first hint I gave you?

In [89]:
# Enter code here:
# the eigenvalues tell you how much variance is explained by each component
pca_model_s_3.explained_variance_ # these are the eigenvalues

array([104.97938107,  39.31877282,  20.61479448])

## Now that you know how much variance is explained by each of the top 3 PCs, let's explore the representation we built further!

#### Let's start by coloring each time point as a function of when it appears in the time series.
#### Hint: you're not changing the plot you're changing the color!

In [96]:
# Enter code here:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')

# Enter code here:
pc1 = neural_pca_s_3[:,0]
pc2 = neural_pca_s_3[:,1]
pc3 = neural_pca_s_3[:,2]
time = range(0,robs.shape[1])

scat = ax.scatter3D(pc1, pc2, pc3, c=time)
fig.colorbar(scat)

<IPython.core.display.Javascript object>

## What does this tell you about the representation?

- the "arm" on the left contains early timepoints,
- the "arm" on the right contains later timepoints
- these likely still correspond to sex, as sex is split by early and late timepoints (according to the imshow we did earlier)

## Now make three separate plots colored by the attack variable! 😡🐭❓

### For plot 1: Plot only the attack data points
### For plot 2: Plot only the other data points
### For plot 3: Plot the attack data points on top of the other data points

#### Hint: it may be easier to separate your data by labels first!
#### Hint: for plot 3 play with opacity (goes by a different name though!), and zorder.

In [102]:
attack.shape

(1, 18561)

In [114]:
# Enter Code Here:
# Enter code here:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')

# timepoints for only attack
attacktimes = np.where(attack[0,:] == 1)[0]
print(attacktimes.shape)

# Enter code here:
pc1 = neural_pca_s_3[attacktimes,0]
pc2 = neural_pca_s_3[attacktimes,1]
pc3 = neural_pca_s_3[attacktimes,2]

atime = range(0,len(attacktimes))

scat = ax.scatter3D(pc1, pc2, pc3, c=atime)
fig.colorbar(scat)
plt.title("Attack Times")
plt.show()

<IPython.core.display.Javascript object>

(1107,)


In [115]:
# Enter Code Here:
# Enter code here:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')

# timepoints for only attack
nonattacktimes = np.where(attack[0,:] == 0)[0]
print(nonattacktimes.shape)

# Enter code here:
pc1 = neural_pca_s_3[nonattacktimes,0]
pc2 = neural_pca_s_3[nonattacktimes,1]
pc3 = neural_pca_s_3[nonattacktimes,2]

ntime = range(0,len(nonattacktimes))

scat = ax.scatter3D(pc1, pc2, pc3, c=ntime)
fig.colorbar(scat)
plt.title("Non-Attack Times")
plt.show()

<IPython.core.display.Javascript object>

(17454,)


In [119]:
# Enter Code Here:
# Enter code here:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')

# Enter code here:
apc1 = neural_pca_s_3[attacktimes,0]
apc2 = neural_pca_s_3[attacktimes,1]
apc3 = neural_pca_s_3[attacktimes,2]

npc1 = neural_pca_s_3[nonattacktimes,0]
npc2 = neural_pca_s_3[nonattacktimes,1]
npc3 = neural_pca_s_3[nonattacktimes,2]

scat = ax.scatter3D(apc1, apc2, apc3, c=atime, zorder=2)
# make the non-attack timepoints more transparent
scat = ax.scatter3D(npc1, npc2, npc3, c=ntime, zorder=1, alpha=0.1)
plt.title("Non-Attack Times")
plt.show()

<IPython.core.display.Javascript object>

## Now build the same plot but color based on the mouse sex variable! ❤️🐭❓

In [121]:
# Enter Code Here:
# Enter Code Here:
# Enter code here:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')

# Enter code here:
pc1 = neural_pca_s_3[:,0]
pc2 = neural_pca_s_3[:,1]
pc3 = neural_pca_s_3[:,2]

scat = ax.scatter3D(pc1, pc2, pc3, c=sex)
fig.colorbar(scat)
plt.title("Times, colors by Sex")
plt.show()

<IPython.core.display.Javascript object>

In [155]:
# turn off interactive plot
%matplotlib notebook

## Great! Now that you know more about the data and PCA, I want you to repeat everything you just did if you reduce the data to 2 PCs! More explicitly:

#### ✦ Train a model on the neural data with 2 PCs.
#### ✦ How much explained variance do these 2 PCs capture? Do you notice anything interesting about these 2 PCs? 😉
#### ✦ How is time visualized in these 2 PCs?
#### ✦ How is 🐭 😡 visualized in these 2 PCs?
#### ✦ How is 🐭 ❤️ visualized in these 2 PCs?

### Finally, if you needed to build a model to classify time, attack, or the visitor's sex how many PCs would you use? Do you lose anything between 3 PCs and 2PCs?

In [156]:
# Enter Code Here:
# make a PCA model with S = 3
pca2 = PCA(n_components=2)

# with the PCA model instance we created to our neural data
pcs2 = pca2.fit_transform(robs.T)
pcs2.shape

(18561, 2)

In [157]:
# plot it
fig = plt.figure()
ax = plt.axes()
ax.scatter(pcs2[:,0], pcs2[:,1])
plt.show()

<IPython.core.display.Javascript object>

In [158]:
# plot it
plt.ioff()
fig = plt.figure()
ax = plt.axes()
ax.scatter(pcs2[:,0], pcs2[:,1], c=attack)
plt.title("Colored by Attack")
plt.show()

<IPython.core.display.Javascript object>

In [162]:
# plot it, with interactivity off
# but turning it off doesn't seem to work...
with plt.ioff():
    fig = plt.figure()
    plt.scatter(pcs2[:,0], pcs2[:,1], c=sex)
    plt.title("Colored by Sex")
    plt.show()

<IPython.core.display.Javascript object>

In [163]:
### Finally, if you needed to build a model to classify time, attack, or the visitor's sex how many PCs would you use? Do you lose anything between 3 PCs and 2PCs?

# sex only needs 2 PCs, but
# attack needs 3 to capture the difference

# Implement PCA yourself!!

## First, mean center your data!
##### The reason we didn't have to do this before is because the PCA function we called automatically did this for us 😱

In [164]:
# Enter code here:
# calculate the mean and subtract it from the data (to move it to the center)
centered_robs = robs - np.mean(robs)

#### Now verify that the pca function returns the same thing in 2D for the not mean centered data and the mean centered data to prove that function we call automatically does this for us. Color using the time steps!

In [168]:
# Enter Code Here:
pca2d0 = PCA(n_components=2)
res0 = pca2d0.fit_transform(robs)

pca2d1 = PCA(n_components=2)
res1 = pca2d1.fit_transform(centered_robs)

In [181]:
# check that the results are the same between centered and not centered
np.sum(res1 - res0)
# the sum is very close to 0, meaning they are the same

1.919020498064583e-13

## Now that we have mean centered data we can transform our data via two paths:
### (1) By stepping through linear decomposition ourselves.
### (2) By using singular value decomposition (a.k.a SVD).

#### Let's start with path (1):
#### First calculate the covariance matrix of your mean centered data.

In [184]:
# Enter code here:
cov = np.cov(robs)

#### Compute the eigenvalues and eigenvectors of the covariance matrix.
##### Hint: Make sure you've sorted the eigenvalues and eigenvectors to be in either ascending or descending order!

In [236]:
# Enter code here:
eigenvalues, eigenvectors = np.linalg.eig(cov)

# sort the eigen[values|vectors] in descending order, by eigenvalues
vals, vecs = list(zip(*sorted(zip(eigenvalues, eigenvectors))))
# convert from tuples to vec
vals = list(vals)
vals.reverse() # in place, lame, but OK
vecs = list(vecs)
vecs.reverse()

# convert to numpy arrays for ease of math
vals = np.array(vals)
vecs = np.array(vecs)

plt.plot(vals)
plt.show()

vals.shape, vecs.shape

<IPython.core.display.Javascript object>

((115,), (115, 115))

#### Now project your mean centered data into a reduced space using the 2 largest eigenvectors.

In [237]:
# Enter code here:
dim1 = vecs[:,0].T @ centered_robs
dim2 = vecs[:,1].T @ centered_robs

#### Plot your transformed data using time as your color!

In [238]:
# Enter code here:
plt.scatter(dim1, dim2, c=time)
plt.show()

<IPython.core.display.Javascript object>

#### Ok but why is the representation flipped?!
#### PCA is sign invariant, meaning that we can multiply the axes by -1 and the interpretation of the dimensionality reduced space stays the same. 
#### Now that we know this is true, change your plot to look like the plot when we use the PCA library directly.

In [239]:
plt.scatter(dim1, -1*dim2, c=time)
plt.show()

<IPython.core.display.Javascript object>

## Great Job!! Now you've calculated PCA all by yourself using matrix operations 🤩🤩🤩 Let's move on to implementing PCA using SVD (singular vector decomposition).

#### Singular vector decomposition is a method that decomposes a matrix into three matricies. U, S, and Vt. The left singular vectors are the columns of U. S are the singular values. V is a matrix whose columns are the right singular vectors. Vt is the transpose of V. Our input data (call it X) equals U\*S\*Vt ➡️ X = U\*S\*Vt.

#### We're not going to implement svd ourselves. Please run np.linalg's svd on our mean centered data.
##### Hint: have we already loaded svd?
##### Hint Hint: run with full_matrices = False otherwise it might take you a while!

In [259]:
# Enter code here:
# we need to transpose the data to do this correctly...
u,s,vh = np.linalg.svd(centered_robs.T, full_matrices=False)

#### What are the dimensions of U, S, and Vt? 

In [260]:
# Enter code here:
u.shape, s.shape, vh.shape

((18561, 115), (115,), (115, 115))

#### Because we used the data we want transformed to calculate U, S, and Vt, we can directly multiply our left singular vectors and singular values together to get our transformed data.

#### Hint: the singular values need to be converted into a diagonal matrix to make the matrix multiplication easier.
#### Hint Hint: We only want to transform our data to a reduced dimension of 2!

In [261]:
# Enter Code Here:
projected_robs = u[:,0:2] @ np.diag(s[0:2])
projected_robs.shape

(18561, 2)

#### Plot your transformed data to match our PCA library plot.

In [263]:
# Enter code here:
# TODO: this is slightly off from the real answer, 
#       and I am not exaclty sure why... 
plt.scatter(-1*projected_robs[:,0], projected_robs[:,1], c=time)
plt.show()

<IPython.core.display.Javascript object>

# To dive deeper into the math behind PCA & SVD stay tuned for day 3!!